In [1]:
import numpy as np
from sklearn.tree import DecisionTreeRegressor

In [2]:
dataFn = "slice_localization_data"
dataTrain = np.genfromtxt(dataFn + "_train.csv", delimiter = ",")
dataTest = np.genfromtxt(dataFn + "_test.csv", delimiter = ",")

In [3]:
nTrain = dataTrain.shape[0]
nTest = dataTest.shape[0]
nOOS = int(min(nTrain / 2, 5e3))
d = dataTrain.shape[1] - 1

np.random.seed(123)
idxOOS = np.random.choice(nTrain, nOOS, replace = False)
XTrain = np.delete(dataTrain[:, :d], idxOOS, axis = 0)
XOOS = dataTrain[idxOOS, :d]
XTest = dataTest[:, :d]
yTrain = np.delete(dataTrain[:, d], idxOOS)
yOOS = dataTrain[idxOOS, d]
yTest = dataTest[:, d]

nTrain = nTrain - nOOS

In [4]:
depths = range(3, 20)
OOSScores = []
for i in depths:
    regTree = DecisionTreeRegressor(max_depth = i, max_features = 100)
    regTree.fit(XTrain, yTrain)
    yOOSPred = regTree.predict(XOOS)
    OOSScores.append(np.sqrt(np.square(yOOSPred - yOOS).sum() / nOOS))
    if np.argmin(OOSScores) == len(OOSScores) - 1:
        regTreeBest = regTree
    else:
        break

In [5]:
yTestPred = regTreeBest.predict(XTest)
np.square(yTest - yTestPred).sum()

856426.2649620692

In [6]:
OOSScores

[13.979366127648078,
 11.939789268556867,
 11.297408087502038,
 10.087056162890981,
 9.62550867471987,
 7.25251579669153,
 7.0269638846175955,
 6.059850104208275,
 4.9603112683922115,
 4.6344862768376585,
 4.202281003601542,
 4.126243162024182,
 3.7096649598663043,
 3.4382698729155674,
 3.824831261676037]

In [11]:
regTreeBest.n_features_in_

379

In [13]:
(regTreeBest.feature_importances_ > 0).sum()

361